In [5]:
import os, sys

sys.path.append(os.path.abspath(os.path.join("../src/")))
from models.Encoder import myself_net, make_myself_net, Block, Bottleneck

from torchinfo import summary

import torch
from torch.cuda.amp import autocast

# input
in_height = 224
in_width = 304
input_rgb = torch.randn(1, 3, in_height, in_width).type(torch.HalfTensor).cuda()
input_dep = torch.randn(1, 1, in_height, in_width).type(torch.HalfTensor).cuda()

# net
# net = make_myself_net("BasicBlock", [8, 16, 32, 64, 128]).cuda()
net = make_myself_net("BasicBlock", [16, 32, 64, 128, 256]).cuda()
# net = make_myself_net("BasicBlock", [32, 64, 128, 128, 128]).cuda()
# net = make_myself_net("BasicBlock", [64, 128, 256, 256, 256]).cuda()
# net = make_myself_net("Bottleneck", [8, 16, 32, 32, 32]).cuda()
# net = make_myself_net("Bottleneck", [16, 32, 64, 64, 64]).cuda()
# net = make_myself_net("Bottleneck", [32, 64, 128, 128, 128]).cuda()
# net = make_myself_net("Bottleneck", [64, 128, 256, 256, 256]).cuda()

# output
with autocast():
    dense_depth_full, dense_depth_1_2, dense_depth_1_4, dense_depth_1_8, dense_depth_1_16 = net(input_rgb, input_dep)

# summary
net_summary = summary(
    model=net,
    input_size=[input_rgb.shape, input_dep.shape],
    col_names=("input_size", "output_size", "num_params", "kernel_size", "mult_adds"),
    depth=2,
)
print("net_mini total params: %0.6f M" % (net_summary.total_params / 1000000.0))
print(net_summary)

net_mini total params: 3.004842 M
Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
myself_net                               [1, 3, 224, 304]          [1, 1, 224, 304]          --                        --                        --
├─Sequential: 1-1                        [1, 3, 224, 304]          [1, 8, 224, 304]          --                        --                        --
│    └─Conv2d: 2-1                       [1, 3, 224, 304]          [1, 8, 224, 304]          216                       [3, 3]                    14,708,736
│    └─BatchNorm2d: 2-2                  [1, 8, 224, 304]          [1, 8, 224, 304]          16                        --                        16
├─Block: 1-2                             [1, 8, 224, 304]          [1, 16, 224, 304]         --                        --                        --
│    └─Conv2d: 2-3                       [1, 8, 224, 304]      

# DKN

In [2]:
import os, sys

sys.path.append(os.path.abspath(os.path.join("../src/")))
from models.Encoder import up_DKN

from torchinfo import summary

import torch

net = up_DKN(16).cuda()
feature = torch.randn(1, 16, 228, 304).cuda()
depth = torch.randn(1, 1, 228, 304).cuda()
output= net(depth, feature).cuda()

summary(
    net,
    input_size=[(1, 1, 228, 304), (1, 16, 228, 304)],
    col_names=("input_size", "output_size", "num_params", "kernel_size", "mult_adds"),
)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
up_DKN                                   [1, 1, 228, 304]          [1, 1, 228, 304]          --                        3                         --
├─Sequential: 1-1                        [1, 1, 228, 304]          [1, 16, 228, 304]         --                        --                        --
│    └─Conv2d: 2-1                       [1, 1, 228, 304]          [1, 16, 228, 304]         16                        [1, 1]                    1,108,992
│    └─BatchNorm2d: 2-2                  [1, 16, 228, 304]         [1, 16, 228, 304]         32                        --                        32
│    └─LeakyReLU: 2-3                    [1, 16, 228, 304]         [1, 16, 228, 304]         --                        --                        --
├─Sequential: 1-2                        [1, 32, 228, 304]         [1, 9, 228, 304]          --   